# 트위터 API로 연관 키워드 분석하기


### step1 . API호출 : 트위터 API 로 데이터 가져오기
    - API데이터로 데이터 프레임 생성하기

### step2. 추출: 키워드 추출
    - 텍스트 데이터 전처리
     - nlik, konlpy를 이용한 키워드 추출
### step3. 분석: 연관 분석을 이용한 키워드 분석
    - 연관 키워드 추출하기
       - 단어 빈도 추출하기
### step4. 시각화 : 연관 키워드 네트워크 시각화
    - 연관 키워드 네트워크 시각화

In [1]:
#주피터 노트북을 실행한 브라우저에서 바로 그림을 볼 수 있게끔 만드는 것
#브라우저 내부(inline)에 바로 그려지도록 해주는 코드 
%matplotlib inline 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# step1. API 호출: 트위터 API로 데이터 가져오기

### API 데이터로 데이터 프레임 생성하기
- anaconda prompt 혹은 terminal에서 아래와 같이 패키지 설치
- pip install tweepy

In [2]:
!pip install tweepy==3.10.0 #트위피 버전 선택해서 설치 

In [37]:
import tweepy
tweepy.__version__

'3.10.0'

In [61]:
# 발급 완료된 키 입력
CONSUMER_KEY = "LUCHdCDyarrHrZtB3TphjDPDX"
CONSUMER_SECRET="cEVd1dTmutWmiISeXlIk7UNZ5qbRlRVJVMg9l4Elvz5GU8doAd"
ACCESS_TOKEN_KEY="169786640-GXkvSASVzzWCVgIgEtO91ENvZCEE4iGIFde9MDyW"
ACCESS_TOKEN_SECRET="xC6ghbPWdEKBmBE8Oi6cVdgVVsRzFpmFLINHLcOksa4QA"

#개인정보 인증을 요청하는 Handler
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

#인증 요청을 수행
auth.set_access_token(ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET)

#트위터 API 사용하기 위한 준비
api = tweepy.API(auth, wait_on_rate_limit=True)

### '손흥민' 키워드 검색

In [62]:
#twitter API 를 사용하여 '손흥민'이 포함된 트윗들을 크롤링한 뒤, 'user_mentions'와 'hashtags' 속성값 추출
keyword = "방탄"

#특정 키워드 크롤링은 search()함수 수행, 결과가 리스트 형태로 출력 
tweets = api.search(keyword)

#entities() 함수로 특정 속성의 메타데이터에 접근하여 정보 추출
for tweet in tweets:
    print(tweet.text)
    print(tweet.entities['user_mentions']) #사용자 댓글
    print(tweet.entities['hashtags']) #해시태그
    print(tweet.created_at) #트윗이 생성된 UTC 시간 

RT @jennycrappy: 윤기야… 우리 아빠도 방탄로고 알아…🙃 애들 우주댓슈스라고 누가 좀 알려주실분?ㅋㅋㅋㅋㅋㅋㅋ

the one and only song of the year is #BTS_Butter by #bts. no other ca…
[{'screen_name': 'jennycrappy', 'name': 'emily💜삶의활력소년단', 'id': 1348221894764285953, 'id_str': '1348221894764285953', 'indices': [3, 15]}]
[{'text': 'BTS_Butter', 'indices': [107, 118]}, {'text': 'bts', 'indices': [122, 126]}]
2022-02-04 05:29:25
RT @JJUNGKOOK_mona: 짜파구리에는 고기가 들어가야된다며 부담스럽겠지라 말하는 석진이 말에 또 바로 냉동실 열어보는 윤기ㅠㅠㅠ그러고는 바로 소고기 맛있게 구워서 동생들이랑 맛있게 먹었쨔나😭우리 방탄의 아빠엄마 맏형들💜

Listening…
[{'screen_name': 'JJUNGKOOK_mona', 'name': '꾹이모나_밤🐰🐕\u200d🦺', 'id': 1244297329319890945, 'id_str': '1244297329319890945', 'indices': [3, 18]}]
[]
2022-02-04 05:29:23
RT @Olivia614: 🌂
방탄소년단 좌우명
무대를 달라
- SUGA
💙
- 21FESTA 우리가쓰는 방탄 프로필

#Butter #BestMusicVideo #iHeartAwards
@BTS_twt
[{'screen_name': 'Olivia614', 'name': 'Olivia⁷ _🍀🍀6 ⟭⟬ ⟬⟭', 'id': 1107756679314067456, 'id_str': '1107756679314067456', 'indices': [3, 13]}, {'screen_name': 'BTS_twt', 'name': '방탄소년단', 'id': 3

### 데이터 프레임 형태로 수집

In [63]:
#크롤링한 데이터를 저장할 데이터 프레임
columns = ['created', 'tweet_text']
df = pd.DataFrame(columns = columns)

#크롤링 수행할 갯수 입력 
max_tweets =100

#Cursor 객체를 사용하여 크롤링 수행
searched_tweets = [status for status in tweepy.Cursor(api.search, q=keyword).items(max_tweets)]



In [64]:
#키워드가 포함된 1000개의 트윗들에서 'text', 'created_at' 정보를 데이터 프레임으로 저장 

for tweet in searched_tweets:
    tweet_json = tweet._json
    tweet_text = tweet_json['text']
    created = tweet_json['created_at']
    row = [created, tweet_text]
    series = pd.Series(row, index=df.columns) #created, tweet_text 시리즈를 생성
    df = df.append(series, ignore_index=True) #생성된 시리즈를 데이터프레임에 추가 

df #데이터 프레임 출력 

,created,tweet_text
0,Fri Feb 04 05:29:25 +0000 2022,RT @jennycrappy: 윤기야… 우리 아빠도 방탄로고 알아…🙃 애들 우주댓슈...
1,Fri Feb 04 05:29:23 +0000 2022,RT @JJUNGKOOK_mona: 짜파구리에는 고기가 들어가야된다며 부담스럽겠지라...
2,Fri Feb 04 05:29:20 +0000 2022,RT @Olivia614: 🌂\n방탄소년단 좌우명\n무대를 달라\n- SUGA\n💙...
3,Fri Feb 04 05:29:18 +0000 2022,RT @BTS_twt: 우리이쁜탄이 연탄이방탄이~~~ 우쮸쮸 https://t.co...
4,Fri Feb 04 05:29:10 +0000 2022,탑꾸 스티커탑꾸 탑꾸스티커 탑로더 판매 양도 포카 포토카드 슬리브 홀로그램 바인더 ...
...,...,...
95,Fri Feb 04 05:18:42 +0000 2022,RT @fourcar_12: 🥰주접 스티커 도안 공유🥰\n\n다이어리 꾸미기 다꾸 ...
96,Fri Feb 04 05:18:40 +0000 2022,RT @cafemartinique: 호석이의 생일을 맞아🐿\n생일이벤트를 준비합니다...
97,Fri Feb 04 05:18:37 +0000 2022,RT @BTS_twt: 방탄생일 축하해 주셔서 감사해요\n저도 작은 선물 준비했어요...
98,Fri Feb 04 05:18:32 +0000 2022,방탄소년단 아미 6기 키트 양도합니다\n개별 판매⭕️ 일괄 판매⭕️\nEach / ...


In [65]:
df.head()

,created,tweet_text
0,Fri Feb 04 05:29:25 +0000 2022,RT @jennycrappy: 윤기야… 우리 아빠도 방탄로고 알아…🙃 애들 우주댓슈...
1,Fri Feb 04 05:29:23 +0000 2022,RT @JJUNGKOOK_mona: 짜파구리에는 고기가 들어가야된다며 부담스럽겠지라...
2,Fri Feb 04 05:29:20 +0000 2022,RT @Olivia614: 🌂\n방탄소년단 좌우명\n무대를 달라\n- SUGA\n💙...
3,Fri Feb 04 05:29:18 +0000 2022,RT @BTS_twt: 우리이쁜탄이 연탄이방탄이~~~ 우쮸쮸 https://t.co...
4,Fri Feb 04 05:29:10 +0000 2022,탑꾸 스티커탑꾸 탑꾸스티커 탑로더 판매 양도 포카 포토카드 슬리브 홀로그램 바인더 ...


In [66]:
#데이터 프레임을 csv로 저장
df.to_csv("tweet_temp.csv", index=False)


# Step2. 추출 : 키워드 추출

## 텍스트 데이터 전처리
- 한글 문자열을 기준으로 키워드 추출 수행
- tweet_text 데이터에서 한글 문자열만을 추출하여 ko_text 생성

In [70]:
import re

#텍스트 정제 함수: 한글 및 띄어쓰기 이외의 문자는 전부 제거
def text_cleaning(text):
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+') #한글의 정규표현식
    result = hangul.sub(' ', text)
    return result

#텍스트 정제함수 text_cleaning() 을 'tweet_text' 피처에 이를 적용, 한글과 띄어쓰기만 추출하여
#ko_text피처 생성
df['ko_text'] = df['tweet_text'].apply(lambda x: text_cleaning(x))
df.head()

,created,tweet_text,ko_text
0,Fri Feb 04 05:29:25 +0000 2022,RT @jennycrappy: 윤기야… 우리 아빠도 방탄로고 알아…🙃 애들 우주댓슈...,윤기야 우리 아빠도 방탄로고 알아 애들 우주댓슈스라고 누가 좀 알려주실분 ㅋㅋㅋㅋ...
1,Fri Feb 04 05:29:23 +0000 2022,RT @JJUNGKOOK_mona: 짜파구리에는 고기가 들어가야된다며 부담스럽겠지라...,짜파구리에는 고기가 들어가야된다며 부담스럽겠지라 말하는 석진이 말에 또 바로 냉동...
2,Fri Feb 04 05:29:20 +0000 2022,RT @Olivia614: 🌂\n방탄소년단 좌우명\n무대를 달라\n- SUGA\n💙...,방탄소년단 좌우명 무대를 달라 우리가쓰는 방탄 프로필
3,Fri Feb 04 05:29:18 +0000 2022,RT @BTS_twt: 우리이쁜탄이 연탄이방탄이~~~ 우쮸쮸 https://t.co...,우리이쁜탄이 연탄이방탄이 우쮸쮸
4,Fri Feb 04 05:29:10 +0000 2022,탑꾸 스티커탑꾸 탑꾸스티커 탑로더 판매 양도 포카 포토카드 슬리브 홀로그램 바인더 ...,탑꾸 스티커탑꾸 탑꾸스티커 탑로더 판매 양도 포카 포토카드 슬리브 홀로그램 바인더 ...


### konlpy 를 이용한 명사 키워드 추출
- 명사만 추출한 후 불용어 및 한글자 키워드 제거


In [77]:
from konlpy.tag import Okt 
from collections import Counter #빈도수 추출하는 라이브러리

#불용어 사전의 텍스트 문서(korean_stopwords.txt) 읽어 stopwords 리스트 생성
korean_stopwords_path = "../data/korean_stopwords.txt"

with open(korean_stopwords_path, encoding = 'utf8') as f: #파일 열기
    stopwords = f.readlines() #파일 읽기 

stopwords =[x.strip() for x in stopwords] #strip()으로 공백 및 줄바꿈 제거


# 품사 중 명사만 추출하여 한글자 키워드 및 불용어 제거
#Okt(Open Korean Text)는 트위터에서 만든 오픈소스 한국어 처리기인 twitter-korean-text
#를 이어받아 만들고 있는 프로젝트 
def get_nouns(x):
    nouns_tagger = Okt() #Okt()클래스 선언
    nouns = nouns_tagger.nouns(x)
    
    #한글자 키워드 제거
    nouns = [noun for noun in nouns if len(noun)>1]
    
    #불용어 제거
    nouns = [noun for noun in nouns if noun not in stopwords]
    
    return nouns #한글자도 아니고 불용어도 아닌 단어들 리턴 

#'ko_text' 피처에 get_nouns() 함수를 적용하여 불용어 및 한글자를 제거한 명사 추출하여 'nouns'피처 생성
df['nouns'] = df['ko_text'].apply(lambda x: get_nouns(x))
print(df.shape) #행과 열의 갯수를 튜플로 반환
df.head()


(100, 4)


,created,tweet_text,ko_text,nouns
0,Fri Feb 04 05:29:25 +0000 2022,RT @jennycrappy: 윤기야… 우리 아빠도 방탄로고 알아…🙃 애들 우주댓슈...,윤기야 우리 아빠도 방탄로고 알아 애들 우주댓슈스라고 누가 좀 알려주실분 ㅋㅋㅋㅋ...,"[윤기, 아빠, 방탄, 로고, 우주, 슈스, 누가]"
1,Fri Feb 04 05:29:23 +0000 2022,RT @JJUNGKOOK_mona: 짜파구리에는 고기가 들어가야된다며 부담스럽겠지라...,짜파구리에는 고기가 들어가야된다며 부담스럽겠지라 말하는 석진이 말에 또 바로 냉동...,"[짜파구리, 고기, 냉동, 어보, 윤기, 소고기, 동생, 먹었쨔, 방탄, 아빠, 엄..."
2,Fri Feb 04 05:29:20 +0000 2022,RT @Olivia614: 🌂\n방탄소년단 좌우명\n무대를 달라\n- SUGA\n💙...,방탄소년단 좌우명 무대를 달라 우리가쓰는 방탄 프로필,"[방탄소년단, 좌우명, 무대, 달라, 방탄, 프로필]"
3,Fri Feb 04 05:29:18 +0000 2022,RT @BTS_twt: 우리이쁜탄이 연탄이방탄이~~~ 우쮸쮸 https://t.co...,우리이쁜탄이 연탄이방탄이 우쮸쮸,"[연탄, 방탄, 우쮸쮸]"
4,Fri Feb 04 05:29:10 +0000 2022,탑꾸 스티커탑꾸 탑꾸스티커 탑로더 판매 양도 포카 포토카드 슬리브 홀로그램 바인더 ...,탑꾸 스티커탑꾸 탑꾸스티커 탑로더 판매 양도 포카 포토카드 슬리브 홀로그램 바인더 ...,"[스티커, 스티커, 로더, 판매, 양도, 포카, 포토, 카드, 슬리브, 홀로그램, ..."


# step3 분석 : 연관 분석을 이용한 키워드 분석

### 연관 키워드 추출하기
### 연관 규칙
- 연관 규칙[아프리오리, Apriori 알고리즘] 
- 연관 규칙은 비지도학습으로 대규모 거래 데이터로부터 함께 구매될 규칙을 도출하여 고객이 특정 상품 구매시 이와 연관성이 높은 상품을 추천하는 것
- Apriori 알고리즘 : 간단한 성능 측정치를 이용해 거대한 DB에서 데이터간의 연관성을 찾는 알고리즘
### 연관규칙에서 사용하는 3가지 통계척도
1. 지지도 (support) : 특정 아이템이 데이터에서 발생하는 빈도
2. 신뢰도 (confidence) : 두 아이템의 연관규칙이 유용한 규칙일 가능성의 척도
3. 향상도 (lift) : 두 아이템의 연관 규칙이 우연인지 아닌지를 나타내는 척도


### Apriori 라이브러리 설치

In [78]:
!pip install apriori apyori

  Created wheel for apriori: filename=apriori-1.0.0-py3-none-any.whl size=2477 sha256=2e3d87b5ea65808e24ce7f5a2c404e372c9d6c1974cc7f8634d206509224dce1
  Stored in directory: c:\users\yj\appdata\local\pip\cache\wheels\e5\13\06\4f2e9149db374a829b7ee97bad99c9abffbf30976a81779b1c
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=69e3fdbc3c4bf7236bac8c533042cfb12750b8e0d17aa4a34e8da6618a8450f6
  Stored in directory: c:\users\yj\appdata\local\pip\cache\wheels\32\2a\54\10c595515f385f3726642b10c60bf788029e8f3a1323e3913a
Successfully built apriori apyori


In [81]:
#연관 분석 연습
from apyori import apriori

transactions = [
    ['손흥민', '시소코'],
    ['손흥민', '케인'],
    ['손흥민', '케인', '포체티노']
]

#연관 분석을 수행
results = list(apriori(transactions))
for result in results:
    print(result)


RelationRecord(items=frozenset({'손흥민'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'손흥민'}), confidence=1.0, lift=1.0)])
RelationRecord(items=frozenset({'시소코'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'시소코'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'케인'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'케인'}), confidence=0.6666666666666666, lift=1.0)])
RelationRecord(items=frozenset({'포체티노'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'포체티노'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'시소코', '손흥민'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'시소코', '손흥민'}), confidence=0.3333333333333333, lift=1.0), Or

In [82]:
# 지지도 0.5, 신뢰도 0.6, 향상도1.0 이상이면서 (손흥민, 케인) 처럼 규칙의 크기가 2이하인 규칙을 추출
list(apriori(transactions,
            min_support=0.5,
            min_confidence=0.6,
            min_lift=1.0,
            max_length=2))

[RelationRecord(items=frozenset({'손흥민'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'손흥민'}), confidence=1.0, lift=1.0)]),
 RelationRecord(items=frozenset({'케인'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'케인'}), confidence=0.6666666666666666, lift=1.0)]),
 RelationRecord(items=frozenset({'케인', '손흥민'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'케인', '손흥민'}), confidence=0.6666666666666666, lift=1.0), OrderedStatistic(items_base=frozenset({'손흥민'}), items_add=frozenset({'케인'}), confidence=0.6666666666666666, lift=1.0), OrderedStatistic(items_base=frozenset({'케인'}), items_add=frozenset({'손흥민'}), confidence=1.0, lift=1.0)])]

### 트위터 연관 키워드 분석

In [84]:
# 트랜젝션 데이터를 추출
transactions = df['nouns'].tolist() #데이터 시리즈를 리스트형태로 변환 
print(transactions)

[['윤기', '아빠', '방탄', '로고', '우주', '슈스', '누가'], ['짜파구리', '고기', '냉동', '어보', '윤기', '소고기', '동생', '먹었쨔', '방탄', '아빠', '엄마', '맏형'], ['방탄소년단', '좌우명', '무대', '달라', '방탄', '프로필'], ['연탄', '방탄', '우쮸쮸'], ['스티커', '스티커', '로더', '판매', '양도', '포카', '포토', '카드', '슬리브', '홀로그램', '바인더', '콜북', '엔시', '몬스타', '엑스', '방탄', '투바', '공포', '일괄', '사진', '참고', '사항', '디엠'], ['민윤기', '칭찬', '감옥', '방탄', '슈가', '투명', '포카', '프레임', '도안', '나눔', '신경', '스냅', '기준', '사이드', '참고', '저장', '편집', '미리보기', '확인'], ['어쨌든', '결과', '하자', '하자', '남준', '방탄'], ['사랑', '방탄', '아미', '아미', '방탄'], ['뮤비', '역시', '방탄'], ['방탄소년단', '매력', '포인트', '정말', '방탄', '프로필'], ['방탄', '방탄', '남자'], ['제이', '어제', '오늘', '캔각', '계속', '방탄'], ['박수영', '레드', '벨벳', '더보이즈', '이주연', '재현', '남돌', '증명사진', '에이', '마이걸', '소녀', '해찬', '찬희', '재윤', '지민', '백현', '세훈', '슈가', '아미', '뉴이스트', '스키', '커미션', '에버', '로우', '도안', '커미션', '여자아이'], ['짱구', '흰둥이', '방탄'], ['방탄', '클라우드', '베리', '포카', '판매', '일괄', '희귀', '남준', '윤기', '호석', '지민', '태형', '정국', '슈가', '알엠', '제이'], ['노트', '방탄'], ['방탄소년단', '방방콘', '미니', '포카', '양도', '디엠', 

In [86]:
#지지도 0.05, 신뢰도 0.1, 향상도 5 이상인 것을 추출
list(apriori(transactions,
            min_support=0.05,
            min_confidence=0.1,
            min_lift=5,
            max_length=2))

[RelationRecord(items=frozenset({'냉동', '고기'}), support=0.06, ordered_statistics=[OrderedStatistic(items_base=frozenset({'고기'}), items_add=frozenset({'냉동'}), confidence=1.0, lift=16.666666666666668), OrderedStatistic(items_base=frozenset({'냉동'}), items_add=frozenset({'고기'}), confidence=1.0, lift=16.666666666666668)]),
 RelationRecord(items=frozenset({'고기', '동생'}), support=0.06, ordered_statistics=[OrderedStatistic(items_base=frozenset({'고기'}), items_add=frozenset({'동생'}), confidence=1.0, lift=16.666666666666668), OrderedStatistic(items_base=frozenset({'동생'}), items_add=frozenset({'고기'}), confidence=1.0, lift=16.666666666666668)]),
 RelationRecord(items=frozenset({'맏형', '고기'}), support=0.06, ordered_statistics=[OrderedStatistic(items_base=frozenset({'고기'}), items_add=frozenset({'맏형'}), confidence=1.0, lift=16.666666666666668), OrderedStatistic(items_base=frozenset({'맏형'}), items_add=frozenset({'고기'}), confidence=1.0, lift=16.666666666666668)]),
 RelationRecord(items=frozenset({'먹었쨔', '고기